
### How LangGraph Works
The framework controls the execution flow by moving data through three primary concepts:


State: This acts as the agent's working memory throughout the process. It stores inputs, outputs, and intermediate results. The state persists across node transitions, enabling complex multi-step reasoning.

Edges: These define the execution order and control the movement between nodes. Transitions can be direct or conditional, which determines the agent's next action and helps manage complex workflows.

Nodes: These are the individual processing units and the primary building blocks of agent graphs.
State represents the agent’s working memory. It stores inputs, outputs, and intermediate results. This state is passed between nodes and updated by each one.

### Defining the State (Working Memory)
State represents the agent’s working memory. It stores inputs, outputs, and intermediate results. This state is passed between nodes and updated by each one

In [ ]:
# Setup and State Management
# A dict or TypeDict is used to hold state
from typing import TypedDict, Annotated, Sequence
import operator

# Define the state structure. It persists across node transitions.
class AgentState(TypedDict):
    messages: Annotated[Sequence[str], operator.add]
    current_plan: str
    tool_results: dict

### Defining Tools
Agents can use external tools like APIs, databases, or calculators. The agent decides when to invoke these tools, and the results are added back to the state.

In [2]:
# Cell 2: Tool Definition
def search_database(query: str) -> str:
    """Mock tool to simulate a database search."""
    return f"Results for {query}: [Data found]"


def calculator(expression: str) -> str:
    """Mock tool to simulate a calculator."""
    return f"Calculated result for {expression}"


# List of tools available to the agent
tools = [search_database, calculator]

### Defining Nodes (Processing Units)
Nodes are the building blocks of the agent graph. Each node is an individual processing unit that performs a specific task, such as reasoning, evaluating, or tool use.

In [3]:
# Node Definitions
# Nodes are functions that call and LLM, execute tools, or transform data
# Each node receives state and returns updated state
def planner_node(state: AgentState):
    """One agent plans the overall task (Planner-Executor Pattern)."""
    # Logic to generate a plan based on state['messages']
    plan = "1. Search database. 2. Calculate results."
    return {"current_plan": plan}

def executor_node(state: AgentState):
    """Another agent executes the plan."""
    # Logic to execute the plan and use tools
    result = search_database("sample query")
    return {"tool_results": {"search": result}}

def human_review_node(state: AgentState):
    """Human-in-the-loop: Humans can approve or reject decisions."""
    # Logic to pause and wait for human input
    approval = input("Approve tool execution? (y/n): ")
    return {"messages": ["Human approved" if approval == 'y' else "Human rejected"]}

### Defining Edges and Compiling the Graph
Edges control the movement between nodes and define the execution order. Transitions can be direct or conditional, helping manage complex workflows.

Define transitions between nodes
* Direct edges → always go to next node
* Conditional edges → routing logic decides next step


In [4]:
# Graph Construction and Edges
from langgraph.graph import StateGraph, END

# Initialize the graph with the state schema
workflow = StateGraph(AgentState)

# Add nodes to the graph
workflow.add_node("planner", planner_node)
workflow.add_node("executor", executor_node)
workflow.add_node("human_review", human_review_node)

# Define transitions (Edges)
workflow.set_entry_point("planner")
workflow.add_edge("planner", "human_review")  # Direct transition
workflow.add_edge("human_review", "executor")
workflow.add_edge("executor", END)  # End of workflow

# Compile the graph
agent_app = workflow.compile()

print("agent app is finished")

agent app is finished


### Execution
Agentic AI relies on these compiled graphs to run long-running tasks and improve overall automation.

In [ ]:
# Running the Agent
initial_state = {
    "messages": ["Analyze the Q3 financial data."],
    "current_plan": "",
    "tool_results": {},
}

# Execute the graph
for output in agent_app.stream(initial_state):
    print("--- Step Output ---")
    print(output)

--- Step Output ---
{'planner': {'current_plan': '1. Search database. 2. Calculate results.'}}
